In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install -r /content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet/requirements.txt
!pip install pytorch-lightning
!pip install wandb



  Using cached torch-1.5.1-cp37-cp37m-manylinux1_x86_64.whl (753.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.1 which is incompatible.
pytorch-lightning 1.5.8 requires torch>=1.7.*, but you have torch 1.5.1 which is incompatible.
  Using cached torch-1.10.1-cp37-cp37m-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [4]:
!pip3 install pickle5

In [5]:
pip install -U PyYAML

  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
    Uninstalling PyYAML-5.3.1:
      Successfully uninstalled PyYAML-5.3.1


In [28]:
%cd /content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet

/content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet


In [53]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet
import pytorch_lightning as pl
import pickle5 as pickle
import weights_and_biases as wandb

#### Some hyperparameters and settings

In [54]:
CONFIG_FILE_PATH = 'config/sdd_longterm.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'sdd_longterm'  # arbitrary name for this experiment
DATASET_NAME = 'sdd'

TRAIN_DATA_PATH = 'data/SDD/train_longterm.pkl'
TRAIN_IMAGE_PATH = 'data/SDD/train'
VAL_DATA_PATH = 'data/SDD/test_longterm.pkl'
VAL_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 4

#### Load config file and print hyperparameters

In [55]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.002,
 'resize': 0.25,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.8,
 'unfreeze': 100,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [14, 29]}

#### Load preprocessed Data

In [56]:
with open(TRAIN_DATA_PATH, "rb") as fh:
	df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh:
	df_val = pickle.load(fh)

In [57]:
df_train

,trackId,frame,x,y,sceneId,metaId
0,2,6881,17.0,893.5,bookstore_0,0
1,2,6911,31.0,904.0,bookstore_0,0
2,2,6941,63.0,910.5,bookstore_0,0
3,2,6971,98.5,917.5,bookstore_0,0
4,2,7001,134.0,919.5,bookstore_0,0
...,...,...,...,...,...,...
39195,114,10460,736.5,1092.5,nexus_9,1119
39196,114,10490,756.0,1084.5,nexus_9,1119
39197,114,10520,775.5,1084.5,nexus_9,1119
39198,114,10550,794.5,1090.5,nexus_9,1119


#### Initiate model and load pretrained weights

In [58]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object'

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [59]:
!wandb login --relogin
wandb.init_wandb(params.copy(), model.model)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


train_loss,▁
train_loss,71448.82812


In [60]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 184/184 [00:00<00:00, 505.21it/s]
/content/drive/MyDrive/Human-Path-Prediction-master/Human-Path-Prediction-master/ynet/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training
184
Epoch 0: 
Val ADE: 263.568115234375 
Val FDE: 203.42698669433594
Best Epoch 0: 
Val ADE: 263.568115234375 
Val FDE: 203.42698669433594


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   0%|          | 1/300 [27:23<136:28:19, 1643.14s/it]

Saved model to: saved_models/Ynet.pt
Epoch 1: 
Val ADE: 195.3909149169922 
Val FDE: 145.2925567626953
Best Epoch 1: 
Val ADE: 195.3909149169922 
Val FDE: 145.2925567626953


...

Saved model to: saved_models/Ynet.pt
Epoch 2: 
Val ADE: 182.4446563720703 
Val FDE: 128.9669952392578
Best Epoch 2: 
Val ADE: 182.4446563720703 
Val FDE: 128.9669952392578


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   1%|          | 3/300 [1:22:09<135:33:43, 1643.18s/it]

Saved model to: saved_models/Ynet.pt


Epoch:   1%|          | 3/300 [1:27:54<145:02:53, 1758.16s/it]


KeyboardInterrupt: ignored

In [ ]:
df.to_csv('Users.csv', sep='\t', index=False,header=True)
